In [1]:
import numpy as np, os, sys
sys.path.append("..")

import torch
from torch import nn
from torch.nn import functional as F 

from src.utils.synthetic_seqdata import download_data, load_data, sequence_string_to_one_hot
from src.models.deepbind_cnn import BasicCNN
from src.trainer.trainer import Trainer
from src.utils.datasets import DNASequenceDataset

import matplotlib as mpl 
from matplotlib import pyplot as plt 
%matplotlib inline

In [2]:
# get data 
savedir = "./data"
_=download_data(savedir)
Xs, Ys = load_data(savedir=savedir)

--2023-07-26 09:58:09--  https://www.dropbox.com/s/drnyowfdv1lbjz6/train_sequences.txt
Resolving www.dropbox.com (www.dropbox.com)... 162.125.4.18, 2620:100:6019:18::a27d:412
Connecting to www.dropbox.com (www.dropbox.com)|162.125.4.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/drnyowfdv1lbjz6/train_sequences.txt [following]
--2023-07-26 09:58:10--  https://www.dropbox.com/s/raw/drnyowfdv1lbjz6/train_sequences.txt
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc1babed41a49823841101268983.dl.dropboxusercontent.com/cd/0/inline/CAkRWSElVHGGCH3rUlq5yW7h752jdleH0Cwv0klZ8M-2ATm7mXWUewbVJxxkdlGl7_B_j4jbX_3Umw0AuvE1QhIlC_XNGI4IQZVhXQF6Ljq-8umGgzZUVs4crx4fEEZsUN8/file# [following]
--2023-07-26 09:58:10--  https://uc1babed41a49823841101268983.dl.dropboxusercontent.com/cd/0/inline/CAkRWSElVHGGCH3rUlq5yW7h752jdleH0Cwv0klZ8M-2ATm7mXWUewbVJxxkdlGl7_B_j4jbX_3Umw0AuvE1QhIlC_XNGI4IQZVhXQ

In [3]:
# set up datasets
datasets = {}
for k in Xs:
    datasets[k] = DNASequenceDataset(sequences=Xs[k], labels=Ys[k], alphabet="ACGT")

# set up dataloaders 
batch_size = 32
loaders = {}
for k, dataset in datasets.items():
    loaders[k] = torch.utils.data.DataLoader(dataset, batch_size=batch_size)
    
# set up the model, lossfn, optimizer, trainer 
learning_rate = 1e-3
model = BasicCNN(input_size=4, output_size=1, kernel_size=9)
lossfn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
trainer = Trainer(model, lossfn, optimizer)

# fit model
nepochs = 5
history = trainer.fit(loaders['train'], val_loader=loaders['valid'], nepochs=nepochs)

 --- [Epoch   1/5] --- 
loss : 0.8068, val_loss : 1.0641
 --- [Epoch   2/5] --- 
loss : 0.7224, val_loss : 0.7096
 --- [Epoch   3/5] --- 
loss : 0.6994, val_loss : 0.7052
 --- [Epoch   4/5] --- 
loss : 0.7096, val_loss : 0.6949
 --- [Epoch   5/5] --- 
loss : 0.6981, val_loss : 0.7143


In [5]:
history

{'loss': [0.8067616931243574,
  0.7224147594682703,
  0.6994235518316155,
  0.7096062132484837,
  0.6980896973174456],
 'val_loss': [1.064138724690392,
  0.7095926224239288,
  0.7052065815244403,
  0.6948687462579637,
  0.7143113632050772]}